In [61]:
# imports
# import plotly.express as px
import pandas as pd
import numpy as np
# import seaborn as sns                       # visualisation
# import matplotlib.pyplot as plt             # visualisation
# %matplotlib inline 
# sns.set(color_codes=True)
import requests                            # Might be needed:
#import datetime                            # Might be needed
import json
# import time
# from pprint import pprint

In [ ]:
save_num_ll = 102
save_num_lat = 102
save_num_long = 102
save_num_df_fsq = 102
save_num_df_yelp = 102

In [62]:
#import data from file
df = pd.read_csv('data/station_data_sample.csv')
ll = pd.read_csv('data/lat_long_series.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [113]:
#define a function to return a list of dictionaries with information about the requested city


import os


def station_loci(ll_value):
    ''' a function that takes string lat-long as '0.00,-0.00' as it's only argument, 
    and returns a DataFrame object with the following parameters
    '''
    api_key = os.environ["FOURSQUARE_API_KEY"]
    fields = 'fsq_id,name,location,geocodes,categories,distance,rating,stats,hours,price,popularity'
    fsq_data_url = (f'https://api.foursquare.com/v3/places/search?ll={ll_value}&fields={fields}&radius=1000')
    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }
    fsq_loci_results = requests.get(fsq_data_url, headers=headers).json()

    list_of_dict = []
    for poi in fsq_loci_results['results']:
        categories = poi.get('categories', None)
        cat_id = categories[0]['id'] if len(categories) > 0 else None
        category_name = categories[0]['name'] if len(categories) > 0 else None

        loci_poi_dict = {
              'lat-long': f'{ll_value}'
            , 'cat_id': cat_id
            , 'category_name': category_name
            , 'categories': categories
            , 'name': poi.get('name', None)
            , 'distance': poi.get('distance', None)
            , 'latitude': poi.get('geocodes', {}).get('main', {}).get('latitude', None)
            , 'longitude': poi.get('geocodes', {}).get('main', {}).get('longitude', None)
            , 'address': poi.get('location', {}).get('formatted_address', None)
            , 'popularity': poi.get('popularity', None)
            , 'price': poi.get('price', None)
            , 'open_now': poi.get('hours', {}).get('open_now', None)
            , 'rating': poi.get('rating', None)
            , 'rating_count': poi.get('stats', {}).get('total_ratings', None)
            # , 'address': poi['location']['address']
            # , 'country': poi['location']['country']
            # , 'cross_street': poi['location']['cross_street']
            # , 'locality': poi['location']['locality']
            # , 'region': poi['location']['region']
            # , 'postcode': poi['location']['postcode']
            # , 'timezone': poi['timezone']
            # , 'link': poi['link']
        }
        list_of_dict.append(loci_poi_dict)
    return pd.DataFrame(list_of_dict)

In [119]:
test_ll = station_loci('49.262487,-123.114397')
test_ll.head(2)

,lat-long,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,price,open_now,rating,rating_count
0,"49.262487,-123.114397",13034,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Moii Cafe,57,49.262562,-123.115171,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",0.221702,1.0,False,8.0,55
1,"49.262487,-123.114397",13034,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Marulilu Cafe,96,49.263364,-123.113942,"451W Broadway W (at Cambie St), Vancouver BC V...",0.953849,2.0,False,8.2,111


In [115]:
# create a new column with the coordinate data we need, reformated to str dtype.

#grab just the two columns we need lat and lon
df.dropna(subset=['latitude', 'longitude'], inplace=True)

# creation of a new variable with lon and lat together, label the column header 'll'
df['ll'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)
df = df[df['ll'] != '0.0,0.0']
print(df.shape)
print(df.dtypes)
df.head()

# reference #from Data wrangling exercise

(241, 11)
name          object
slots          int64
empty          int64
available      int64
status        object
bank_card     object
latitude     float64
longitude    float64
timestamp     object
unique_id      int64
ll            object
dtype: object


,name,slots,empty,available,status,bank_card,latitude,longitude,timestamp,unique_id,ll
0,0001 10th & Cambie,35,20,15,online,No,49.262487,-123.114397,2022-11-05T06:56:07.919000Z,1,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-05T06:56:07.922000Z,4,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,26,7,19,online,No,49.279764,-123.110154,2022-11-05T06:56:07.923000Z,5,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),16,8,8,online,No,49.260599,-123.113504,2022-11-05T06:56:07.924000Z,7,"49.260599,-123.113504"
4,0008 8th & Ash,16,13,3,online,No,49.264215,-123.117772,2022-11-05T06:56:07.925000Z,8,"49.264215,-123.117772"


In [116]:
# create a series of ll, confirm format for api url
ll = df['ll']
ll.head(5)

0    49.262487,-123.114397
1    49.274566,-123.121817
2    49.279764,-123.110154
3    49.260599,-123.113504
4    49.264215,-123.117772
Name: ll, dtype: object

In [117]:
# populate an empty list with the pd.df objects from each ll location:
df_list = []
for loci in ll:
    next = station_loci(loci)
    df_list.append(next)

In [118]:
# refresh df objects to avoid duplication
df_fsq = pd.DataFrame()
x = pd.DataFrame()

# generate full df object with all locations through concatenation 
for df_loci in df_list:
    x = pd.concat([df_fsq, df_loci])
    df_fsq = x

In [120]:
#confirm success:
df_fsq.shape

(2410, 14)

In [122]:
#save the new df:
df_fsq.to_csv(f'data/df_fsq_all_loci{save_num_df_fsq}.csv', index=False)
save_num_df_fsq += 1

In [ ]:
df_fsq.head(20)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [128]:
#define a function to return a list of dictionaries with information about the requested city

import requests
import os


def station_loci(lat, long):
    ''' a function that takes string lat and long as '0.00', and '-0.00' as it's arguments, 
    and returns a DataFrame object with the following parameters
    '''
    api_key = os.environ["YELP_API_KEY"]
    yelp_data_url = (f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000')
    payload={}
    headers = {
        'accept': 'application/json',             #Incorporate headers and API key
        'Authorization': f'Bearer {api_key}'
    }
    yelp_loci_results = requests.get(yelp_data_url, headers=headers, data=payload).json()           #Perform get request

    list_of_dict = []
    for poi in yelp_loci_results["businesses"]:
        loci_poi_dict = {
              'lat-long': f'{lat}, {long}'
            , 'category_name': poi['categories'][0]['alias']
            , 'categories': poi['categories']
            , 'name': poi['name']
            , 'distance': poi['distance']
            , 'latitude': poi['coordinates']['latitude']
            , 'longitude': poi['coordinates']['longitude']
            , 'address': poi['location']['display_address']
            , 'price': poi.get('price', None)
            , 'is_closed': poi.get('is_closed')
            # , 'display_phone': poi.get('location']['display_phone')
            , 'rating': poi.get('rating')
            , 'rating_count': poi.get('review_count')
            # , 'address1': poi['location']['address1']
            # , 'country': poi['location']['country']
            # , 'city': poi['location']['city']
            # , 'state': poi['location']['state']
            # , 'zip_code': poi['location']['zip_code']
        }
        list_of_dict.append(loci_poi_dict)
    return pd.DataFrame(list_of_dict)

In [125]:
# a test run of the function
yelp_test_ll = station_loci('49.262487','-123.114397')
yelp_test_ll.head(2)

,lat-long,category_name,categories,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count
0,"49.262487, -123.114397",mexican,"[{'alias': 'mexican', 'title': 'Mexican'}]",La Taqueria Pinche Taco Shop,169.517456,49.263559,-123.112736,"[2450 Yukon Street, Vancouver, BC V5Z 3V6, Can...",$$,False,4.0,667
1,"49.262487, -123.114397",chinese,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",Peaceful Restaurant,146.418009,49.263147,-123.116143,"[110-532 W Broadway, Vancouver, BC V5Z 1E9, Ca...",$$,False,3.5,610


In [126]:
#to run location queries, we'll need the lat and log values for each station to be accessable:

# create a series of latitude:
lat = df['latitude']
print(lat.head(5))
# create a series of longitude:
long = df['longitude']
print(long.head(5))
type(long)

# Save the new series
lat.to_csv(f'data/lat_series{save_num_lat}.csv', index=False)
long.to_csv(f'data/long_series{save_num_long}.csv', index=False)
save_num_lat += 1
save_num_long += 1

0    49.262487
1    49.274566
2    49.279764
3    49.260599
4    49.264215
Name: latitude, dtype: float64
0   -123.114397
1   -123.121817
2   -123.110154
3   -123.113504
4   -123.117772
Name: longitude, dtype: float64


In [129]:
# create a while loop to run the series lat and long through the yelp api function
# This will generate a list of 241 df; one for each of the citybike.s stations

num_rows = df.shape[0]
row_counter = 0
yelp_df_list = []
while row_counter < num_rows:
    next = station_loci(lat[row_counter], long[row_counter])
    yelp_df_list.append(next)
    row_counter += 1

In [130]:
# refresh df objects to avoid duplication
df_yelp = pd.DataFrame()
x = pd.DataFrame()

# generate full df object with all locations through concatenation 
for df_loci in yelp_df_list:
    x = pd.concat([df_yelp, df_loci])
    df_yelp = x

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [131]:
#confirm success:
print(df_yelp.shape)
# df_yelp.head(50)

(4788, 12)


In [133]:
# save new df
df_yelp.to_csv(f'data/df_yelp_all_loci{save_num_df_yelp}.csv', index=False)
save_num_df_yelp += 1

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [ ]:
df_yelp.sort_values(by = ['rating', 'category_name'], ascending = False).head(10)

,lat-long,category_name,name,distance,latitude,longitude,address,is_closed,review_count,rating
18,"49.27033, -123.10275",Street Vendors,Johnny's Pops,1052.246707,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
2,"49.277469, -123.081315",Street Vendors,Johnny's Pops,901.446270,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
2,"49.279369, -123.091241",Street Vendors,Johnny's Pops,255.755370,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
15,"49.283992, -123.091206",Street Vendors,Johnny's Pops,718.673135,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
3,"49.273777, -123.092723",Street Vendors,Johnny's Pops,447.037606,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
3,"49.277595, -123.09583",Street Vendors,Johnny's Pops,152.913685,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
2,"49.279235, -123.087528",Street Vendors,Johnny's Pops,479.817198,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
2,"49.277333, -123.087359",Street Vendors,Johnny's Pops,464.722795,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
1,"49.277615, -123.091654",Street Vendors,Johnny's Pops,151.578373,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
3,"49.280852, -123.084746",Street Vendors,Johnny's Pops,738.004461,49.277771,-123.090302,"[Vancouver, BC V6A, Canada]",False,82,5.0
